In [1]:
import os
import numpy as np
import nibabel as nib
import pandas as pd
import sys
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedGroupKFold, cross_validate
import numpy as np
from sklearn.linear_model import LogisticRegression
from copy import deepcopy
import itertools

# add path to classification analysis functions
from mixed_sigmoid_normalisation import MixedSigmoidScaler

In [2]:
pyspi_res_path = "/headnode1/abry4213/data/Cogitate_MEG_challenge/derivatives/time_series_features"

In [3]:
all_pyspi_res_list = []

for pyspi_res_file in os.listdir(pyspi_res_path):
    pyspi_res = pd.read_csv(f"{pyspi_res_path}/{pyspi_res_file}")
    all_pyspi_res_list.append(pyspi_res)

all_pyspi_res = pd.concat(all_pyspi_res_list)

Starting with the Pearson correlation, comparing each pair of stimulus types during relevant non-target onset period

In [4]:
# Comparisons we care about -- during stimulus presentation
GWNT_to_CS_Pearson_relevant_onset = all_pyspi_res.query("SPI == 'cov_EmpiricalCovariance' & meta_ROI_from == 'GNWT' & meta_ROI_to == 'Category_Selective' & relevance_type == 'Relevant non-target' & stimulus == 'on'").dropna()
IIT_to_CS_relevant_onset = all_pyspi_res.query("meta_ROI_from == 'IIT' & meta_ROI_to == 'Category_Selective' & relevance_type == 'Relevant non-target' & stimulus == 'on'").dropna()

# Only keep SPIs for which there are 180 non-NA values
IIT_to_CS_relevant_onset = IIT_to_CS_relevant_onset[IIT_to_CS_relevant_onset["SPI"].map(IIT_to_CS_relevant_onset["SPI"].value_counts()) == 180]

In [6]:
# List of stimulus types
stimulus_types = all_pyspi_res.stimulus_type.unique().tolist()

# Generate pairs of stimulus types
stimulus_combos = list(itertools.combinations(stimulus_types, 2))

SPI_combo_res_list = []

# Iterate over SPIs
for SPI in IIT_to_CS_relevant_onset.SPI.unique().tolist():

    # Iterate over stimulus combos
    for this_combo in stimulus_combos:

        # # Start with face and object
        # this_combo = ('face', 'object')

        # Extract just GNWT/CS data first
        IIT_to_CS_relevant_onset_this_combo = IIT_to_CS_relevant_onset.query(f"stimulus_type in {this_combo} & SPI == '{SPI}'")

        # Define classification model
        model = LogisticRegression(penalty='l1', C=1, solver='liblinear', random_state=127)
        pipe = Pipeline([('scaler', MixedSigmoidScaler(unit_variance=True)), 
                                ('model', model)])

        # Fit classifier
        X = IIT_to_CS_relevant_onset_this_combo.value.to_numpy().reshape(-1, 1)
        y = IIT_to_CS_relevant_onset_this_combo.stimulus_type.to_numpy().reshape(-1, 1)
        groups = IIT_to_CS_relevant_onset_this_combo.subject_ID.to_numpy().reshape(-1, 1)

        group_stratified_CV = StratifiedGroupKFold(n_splits = 10, shuffle = True, random_state=127)

        this_classifier_res = cross_validate(pipe, X, y, groups=groups, cv=group_stratified_CV, scoring="accuracy", n_jobs=1, 
                                                    return_estimator=False, return_train_score=False)["test_score"].mean()
        
        this_SPI_combo_df = pd.DataFrame({"SPI": [SPI], "stimulus_combo": [this_combo], "accuracy": [this_classifier_res]})

        SPI_combo_res_list.append(this_SPI_combo_df)

SPI_combo_res = pd.concat(SPI_combo_res_list)

/headnode1/abry4213/.conda/envs/pyspi/lib/python3.9/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/headnode1/abry4213/.conda/envs/pyspi/lib/python3.9/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/headnode1/abry4213/.conda/envs/pyspi/lib/python3.9/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/headnode1/abry4213/.conda/envs/pyspi/lib/python3.9/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vecto